### Imports

In [1274]:
import math
import json
import requests
import numpy as np
import pandas as pd
pd.set_option('max_rows', None)

from google.colab import drive
drive.mount('/content/drive')

from google.colab import auth
auth.authenticate_user()

import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

allow_runall = False

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1275]:
votesheet = gc.open('RYM AniChart 4.0').worksheet('Votes')
votes = votesheet.get_all_values()

chartsheet = gc.open('RYM AniChart 4.0').worksheet('Charts')
charts = chartsheet.get_all_values()

In [1276]:
vdf = pd.DataFrame(votes[1:], columns=votes[0])
display(vdf.head())

cdf = pd.DataFrame(charts[1:], columns=charts[0])
cdf.index = cdf["Title"]
cdf.drop("Title", axis=1, inplace=True)
display(cdf.head())

,acchonburike,apodesta,Araby_,Bagman,BananaSam,Bhaos,bluedelirium,CaptainBoomerang,Chnapik,chumlum,ComprehendOrNot,crystal146,darsu,Decibelle,DissapointedSun19,dronengaze,Elio90,goriq,GrimesCuck,gumbyjaws,Happy_New_Year,hopeascendchaos,Illogic93,JessiEye,Jewelz,joels344_3,jpmouz,koybz,Kuramhan,Lan24,leutian,LordOfSushi,Madluke,Magic8Ball,matthewhuynh,meatEatingorchids,MechaGhidorah,MicrosoftDog,MisterMonday,mjf314,monkeyballs__,moonhalo,Nakkinak,nekodoiko,nestingquay,neye,NightlifeCommando,nospr,OutrunCam,ppepperonii,Prio,profty,queezus,RafaElvisTek,Rengar18,Romario3210,Ryoanji,Sarubadooru,Sean_Ruins_Everything,simo000,skeletorissatan,slib,sono17,strictlygenteel,stuie299,suddenlywolf,tbchico8,tezukababe,ThelMi,TheGendoPose,Toadfan64,trimonuter,Triturate,trrt,Vancyon,warioman96,WeriderIsBetter,whoa,Xantha_Page,YasashiiDia
0,1. Ping Pong the Animation,01. Elfen Lied,01. Revolutionary Girl Utena,01. Revolutionary Girl Utena,01. K-On!,01. Neon Genesis Evangelion,01. Cowboy Bebop,01. Revolutionary Girl Utena,01. Serial Experiments Lain,01. Anne of Green Gables,01. Ashita no Joe,01. Serial Experiments Lain,01. Haibane Renmei,01. Girls' Last Tour,01. Neon Genesis Evangelion,01. Neon Genesis Evangelion,01. Cowboy Bebop,01. From the New World,01. Eureka Seven,01. A Dog of Flanders,01. FLCL,01. Neon Genesis Evangelion,01. Steins;Gate,01. Kill La Kill,01. Neon Genesis Evangelion,01. Legend of the Galactic Heroes,01. Cowboy Bebop,01. Aria,01. Revolutionary Girl Utena,01. Revolutionary Girl Utena,01. Cowboy Bebop,01. Neon Genesis Evangelion,01. Natsume's Book of Friends,01. Neon Genesis Evangelion,01. Hunter x Hunter (2011),01. When They Cry,01. Neon Genesis Evangelion,01. Erased,01. Neon Genesis Evangelion,01. Mushishi,01. Space Adventure Cobra!,01. Fullmetal Alchemist,01. Neon Genesis Evangelion,01. Death Note,01. Revolutionary Girl Utena,01. Paranoia Agent,01. Neon Genesis Evangelion,01. Code Geass,01. Kaiji,01. Cowboy Bebop,01. Tamako Market,01. FLCL,01. Cowboy Bebop,01. FLCL,01. Monogatari,01. Anne of Green Gables,01. Heidi: Girl of the Alps,01. Penguindrum,-1. Serial Experiments Lain,1. Mushishi,01. Dragon Ball Z,-1. Anne of Green Gables,01. Penguindrum,01. Eureka Seven,1. Neo Yokio,01. Baccano!,01. Steins;Gate,-1. Elfen Lied,01. Revolutionary Girl Utena,01. Neon Genesis Evangelion,01. One Punch Man,01. A Place Further Than the Universe,01. Turn A Gundam,01. Trigun,01. Clannad,01. Hunter x Hunter (2011),01. Neon Genesis Evangelion,01. Serial Experiments Lain,01. Mushishi,01. Hunter x Hunter (2011)
1,,,02. Neon Genesis Evangelion,02. Serial Experiments Lain,02. Neon Genesis Evangelion,02. Rurouni Kenshin,02. Shigurui,02. Mononoke,02. Mushishi,02. Nichijou,02. Neon Genesis Evangelion,02. School-Live!,02. Dennou Coil,02. A Place Further Than the Universe,02. The Tatami Galaxy,02. Hunter x Hunter (2011),02. Kino's Journey,02. Serial Experiments Lain,02. Revolutionary Girl Utena,02. Future Boy Conan,02. Ping Pong the Animation,02. Hunter x Hunter (2011),02. Puella Magi Madoka Magica,02. One Punch Man,02. The Melancholy of Haruhi Suzumiya,02. Fullmetal Alchemist: Brotherhood,02. Monogatari,02. Kyousougiga,02. Aria,02. Legend of the Galactic Heroes,02. Azumanga Daioh,02. FLCL,02. Kino's Journey,02. Revolutionary Girl Utena,02. March Comes in Like a Lion,,02. Berserk,02. Sailor Moon,02. Death Frenzy,02. Kino's Journey,,02. Fate/Zero,02. Revolutionary Girl Utena,02. Ping Pong the Animation,02. Serial Experiments Lain,02. JoJo's Bizarre Adventure (2012),02. Fullmetal Alchemist: Brotherhood,02. Gurren Lagann,02. Neon Genesis Evangelion,02. Dragon Ball,02. Revue Starlight,02. Cowboy Bebop,02. Revolutionary Girl Utena,02. The Tatami Galaxy,02. March Comes in Like a Lion,02. Revolutionary Girl Utena,02. Hunter x Hunter (1999),02. The Melancholy of Haruhi Suzumiya,-1. Neon Genesis Evangelion,2. The Tatami Galaxy,02. One Piece,-1. Kaiba,02. Gunslinger Girl,02. The Big O,,02. Hunter x Hunter (2011),02. Cowboy Bebop,-1. FLCL,02. Neon Genesis Evangelio

,Rank,Votes,Score,Esoteric\nRank,Esoteric\nScore,Source,Episodes,Studio,First Air Date,Last Air Date,Genres,AniListID,TMDbID,IMGID,AltTitle
Title,,,,,,,,,,,,,,,
Neon Genesis Evangelion,1,61,1374,148,18.1,Original,26,Gainax,1995-10-04,1996-03-27,"Action, Drama, Mecha, Mystery, Psychological, ...",30,890,/y2ah9t0navXyIvoHg1uIbIHO3tt.jpg,
Serial Experiments Lain,2,49,1112,184,12.4,Original,13,Triangle Staff,1998-07-06,1998-09-28,"Drama, Mystery, Psychological, Sci-Fi, Superna...",339,1087,/ej3tcxv2YYVWy6WoOeWZTcrkiI8.jpg,
Cowboy Bebop,3,46,978,188,12.3,Original,26,Sunrise,1998-04-03,1999-04-24,"Action, Adventure, Drama, Sci-Fi",1,30991,/xDiXDfZwC6XYC6fxHI1jl3A3Ill.jpg,
FLCL,4,37,785,226,8.5,Original,6,Gainax,2003-05-05,2003-05-13,"Action, Comedy, Mecha, Sci-Fi",227,5895,/FkgA8CcmiLJGVCRYRQ2g2UfVtF.jpg,
Revolutionary Girl Utena,5,30,708,193,12.1,Original,39,J.C. Staff,1997-04-02,1997-12-24,"Drama, Fantasy, Mystery, Psychological, Romance",440,1042,/bnWxRJHWbYcAGq9HWWuKCwm7I5X.jpg,


### Check for new shows

In [1277]:
titlesdf = vdf.applymap(lambda x: x.split(". ", 1)[-1]).to_numpy().flatten()
titlesdf = np.unique(titlesdf)
titlesdf = pd.DataFrame(titlesdf, columns=["Title"])[1:]

In [1278]:
newtitles = titlesdf["Title"][~titlesdf["Title"].str.lower().isin(cdf.index.str.lower())]
newtitles2 = titlesdf["Title"][~titlesdf["Title"].isin(cdf.index)]

newtitlediff = set(newtitles2) - set(newtitles)
if len(newtitlediff) > 0:
  print(newtitlediff)
  raise Exception("Check capitalization")

cdf = cdf.append(pd.DataFrame(index=newtitles))
newtitles

Series([], Name: Title, dtype: object)

In [1279]:
if len(newtitles) > 0:
  assert(allow_runall)

### Make vote matrix

In [1280]:
cdf = cdf[~cdf.index.duplicated()]
mdf = pd.DataFrame(index = cdf.index, columns = vdf.columns)

for voter in vdf:
  for vote in vdf[voter]:
    if vote == "": break
    rank, title = vote.split(". ", 1)
    mdf.loc[title, voter] = rank

mdf = mdf.astype(pd.SparseDtype("int", np.nan))
print('Density:', mdf.sparse.density, '\nmdf.shape', mdf.shape, "cdf.shape", cdf.shape)
display(mdf.head())

assert(set(mdf.index) - set(cdf.index) == set())
assert(not mdf.index.has_duplicates)
assert(cdf.shape[0] == mdf.shape[0])

Density: 0.053230932203389834 
mdf.shape (472, 80) cdf.shape (472, 15)


,acchonburike,apodesta,Araby_,Bagman,BananaSam,Bhaos,bluedelirium,CaptainBoomerang,Chnapik,chumlum,ComprehendOrNot,crystal146,darsu,Decibelle,DissapointedSun19,dronengaze,Elio90,goriq,GrimesCuck,gumbyjaws,Happy_New_Year,hopeascendchaos,Illogic93,JessiEye,Jewelz,joels344_3,jpmouz,koybz,Kuramhan,Lan24,leutian,LordOfSushi,Madluke,Magic8Ball,matthewhuynh,meatEatingorchids,MechaGhidorah,MicrosoftDog,MisterMonday,mjf314,monkeyballs__,moonhalo,Nakkinak,nekodoiko,nestingquay,neye,NightlifeCommando,nospr,OutrunCam,ppepperonii,Prio,profty,queezus,RafaElvisTek,Rengar18,Romario3210,Ryoanji,Sarubadooru,Sean_Ruins_Everything,simo000,skeletorissatan,slib,sono17,strictlygenteel,stuie299,suddenlywolf,tbchico8,tezukababe,ThelMi,TheGendoPose,Toadfan64,trimonuter,Triturate,trrt,Vancyon,warioman96,WeriderIsBetter,whoa,Xantha_Page,YasashiiDia
Title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Neon Genesis Evangelion,NaN,NaN,2,3,2.0,1.0,5,17,NaN,22.0,2.0,3.0,22.0,14.0,1.0,1,26.0,23.0,NaN,17.0,22.0,1.0,13.0,8.0,1.0,3.0,7,4.0,4,NaN,NaN,1.0,5.0,1.0,3.0,NaN,1.0,NaN,1.0,25,NaN,20.0,1,4.0,16,3.0,1.0,NaN,2.0,NaN,7,4.0,4.0,NaN,33.0,8,NaN,NaN,-1.0,7,NaN,-1.0,3,8.0,NaN,6.0,5.0,-1.0,2.0,1.0,NaN,13.0,7.0,12,NaN,2,1.0,46.0,NaN,3
Serial Experiments Lain,NaN,NaN,11,2,NaN,4.0,6,43,1.0,10.0,3.0,1.0,15.0,8.0,NaN,17,27.0,2.0,9.0,4.0,23.0,4.0,17.0,15.0,NaN,18.0,11,17.0,5,5.0,NaN,5.0,NaN,10.0,11.0,NaN,NaN,NaN,9.0,3,NaN,NaN,12,NaN,2,23.0,NaN,NaN,5.0,NaN,14,NaN,NaN,NaN,11.0,30,NaN,NaN,-1.0,6,NaN,-1.0,5,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,12.0,15.0,3,NaN,9,19.0,1.0,NaN,9
Cowboy Bebop,NaN,NaN,5,5,NaN,NaN,1,37,NaN,NaN,NaN,19.0,NaN,15.0,0.0,3,1.0,NaN,NaN,3.0,NaN,5.0,NaN,NaN,6.0,5.0,1,NaN,6,4.0,1.0,33.0,28.0,NaN,7.0,NaN,NaN,NaN,43.0,5,NaN,NaN,7,NaN,3,11.0,16.0,NaN,NaN,1.0,9,2.0,1.0,3.0,20.0,28,NaN,NaN,-1.0,5,NaN,NaN,14,NaN,NaN,NaN,2.0,NaN,NaN,5.0,3.0,17.0,28.0,5,5.0,12,NaN,4.0,NaN,5
FLCL,NaN,NaN,4,18,NaN,3.0,8,10,NaN,43.0,11.0,9.0,NaN,NaN,NaN,6,19.0,NaN,3.0,21.0,1.0,3.0,NaN,NaN,NaN,NaN,13,16.0,3,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32,NaN,NaN,4,16.0,10,8.0,19.0,NaN,NaN,NaN,18,1.0,NaN,1.0,NaN,3,NaN,NaN,NaN,17,NaN,NaN,12,NaN,NaN,5.0,NaN,-1.0,NaN,NaN,4.0,NaN,NaN,8,NaN,11,NaN,26.0,NaN,49
Revolutionary Girl Utena,NaN,NaN,1,1,3.0,NaN,3,1,NaN,NaN,14.0,NaN,NaN,26.0,NaN,18,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,20.0,22,21.0,1,1.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,41.0,17,NaN,NaN,2,NaN,1,NaN,NaN,NaN,NaN,NaN,5,NaN,2.0,NaN,NaN,2,NaN,NaN,NaN,4,NaN,-1.0,7,NaN,NaN,NaN,NaN,-1.0,1.0,NaN,NaN,NaN,NaN,4,NaN,15,NaN,NaN,NaN,19


###Make df with voter meta data

In [1281]:
# Number of ranked votes
ranked_votes = [max(max(mdf[mdf[col].notna()][col]),0) for col in mdf.columns]

# Number of unranked votes
unranked_votes = [mdf[mdf[col]==-1][col].count() for col in mdf.columns]

# Total votes
from operator import add
total_votes= list(map(add, ranked_votes, unranked_votes))

vm_index = ['ranked_votes','unranked_votes', 'total_votes']
vm_data = [ranked_votes, unranked_votes, total_votes]

vmdf = pd.DataFrame(vm_data, columns=mdf.columns, index=vm_index)
vmdf

,acchonburike,apodesta,Araby_,Bagman,BananaSam,Bhaos,bluedelirium,CaptainBoomerang,Chnapik,chumlum,ComprehendOrNot,crystal146,darsu,Decibelle,DissapointedSun19,dronengaze,Elio90,goriq,GrimesCuck,gumbyjaws,Happy_New_Year,hopeascendchaos,Illogic93,JessiEye,Jewelz,joels344_3,jpmouz,koybz,Kuramhan,Lan24,leutian,LordOfSushi,Madluke,Magic8Ball,matthewhuynh,meatEatingorchids,MechaGhidorah,MicrosoftDog,MisterMonday,mjf314,monkeyballs__,moonhalo,Nakkinak,nekodoiko,nestingquay,neye,NightlifeCommando,nospr,OutrunCam,ppepperonii,Prio,profty,queezus,RafaElvisTek,Rengar18,Romario3210,Ryoanji,Sarubadooru,Sean_Ruins_Everything,simo000,skeletorissatan,slib,sono17,strictlygenteel,stuie299,suddenlywolf,tbchico8,tezukababe,ThelMi,TheGendoPose,Toadfan64,trimonuter,Triturate,trrt,Vancyon,warioman96,WeriderIsBetter,whoa,Xantha_Page,YasashiiDia
ranked_votes,1,1,50,50,10,11,14,50,10,50,38,22,30,50,8,25,30,30,15,44,25,10,26,25,20,30,50,50,50,20,20,50,35,15,30,1,9,11,50,32,1,50,31,17,20,50,50,22,10,5,40,10,15,10,50,50,25,30,0,50,7,0,15,10,1,20,10,0,10,9,10,35,30,20,5,20,33,50,1,50
unranked_votes,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0,17,0,0,0,0,0,13,0,0,0,0,0,0,0,0,0,0,0,0
total_votes,1,1,50,50,10,11,14,50,10,50,38,22,30,50,8,25,30,30,15,44,25,10,26,25,20,30,50,50,50,20,20,50,35,15,30,1,9,11,50,32,1,50,31,17,20,50,50,22,10,5,40,10,15,10,50,50,25,30,11,50,7,17,15,10,1,20,10,13,10,9,10,35,30,20,5,20,33,50,1,50


###Calculate score and rankings

In [1282]:
# Count votes per title
counts = mdf.count(axis=1)
cdf['Votes'] = counts.iloc[:].values

# Modified Borda count
cdf['Score'] = 0
cdf['Esoteric\nScore'] = 0

most_votes = max(cdf['Votes'])
weight = 7
inv_pop_weight = 0.15

for title in cdf.index:
    
    points = 0
    esoteric = 0
    votes = cdf.loc[title,'Votes'] 

    if votes < 1:
        cdf.loc[title,'Score'] = 0
        cdf.loc[title,'Esoteric\nScore'] = 0
        continue
    
    for voter in mdf.columns:
        
        total_votes = vmdf.loc['total_votes',voter]
        ranked_votes = vmdf.loc['ranked_votes',voter]
        rank = mdf.loc[title,voter]
        
        if rank > 0: # Ranked votes
            points += total_votes + 1 - rank
            esoteric += 1 + (total_votes-1)*np.exp(-(rank-1)**2 / (2*weight**2))
            
        elif rank == -1: # Unranked votes
            points += 0.5 * (1+total_votes-ranked_votes)
            esoteric += 1
    
    cdf.loc[title,'Score'] = points
    
    esoteric *= 1 + most_votes * np.exp(-(votes-1)**2 / (2*(inv_pop_weight*most_votes)**2))
    esoteric /= 1 + most_votes
    cdf.loc[title,'Esoteric\nScore'] = esoteric
    
cdf['Score'] += 0.0001*cdf['Votes'] # hacky way of sorting ties by votes for ranking

In [1283]:
cdf['Rank'] = cdf['Score'].rank(ascending=False,method='min').astype(int)
cdf['Score'] = cdf['Score'].round(1)
cdf['Esoteric\nScore'] = cdf['Esoteric\nScore'].round(1)
cdf['Esoteric\nRank'] = cdf['Esoteric\nScore'].rank(ascending=False,method='min').astype(int)
cdf.loc[cdf['Score']<1,'Votes'] = 0
cdf.sort_values(by=['Rank']).head(5)

,Rank,Votes,Score,Esoteric\nRank,Esoteric\nScore,Source,Episodes,Studio,First Air Date,Last Air Date,Genres,AniListID,TMDbID,IMGID,AltTitle
Title,,,,,,,,,,,,,,,
Neon Genesis Evangelion,1,61,1374.0,148,18.1,Original,26,Gainax,1995-10-04,1996-03-27,"Action, Drama, Mecha, Mystery, Psychological, ...",30,890,/y2ah9t0navXyIvoHg1uIbIHO3tt.jpg,
Serial Experiments Lain,2,49,1112.0,184,12.4,Original,13,Triangle Staff,1998-07-06,1998-09-28,"Drama, Mystery, Psychological, Sci-Fi, Superna...",339,1087,/ej3tcxv2YYVWy6WoOeWZTcrkiI8.jpg,
Cowboy Bebop,3,46,978.0,188,12.3,Original,26,Sunrise,1998-04-03,1999-04-24,"Action, Adventure, Drama, Sci-Fi",1,30991,/xDiXDfZwC6XYC6fxHI1jl3A3Ill.jpg,
FLCL,4,37,785.0,226,8.5,Original,6,Gainax,2003-05-05,2003-05-13,"Action, Comedy, Mecha, Sci-Fi",227,5895,/FkgA8CcmiLJGVCRYRQ2g2UfVtF.jpg,
Revolutionary Girl Utena,5,30,708.0,193,12.1,Original,39,J.C. Staff,1997-04-02,1997-12-24,"Drama, Fantasy, Mystery, Psychological, Romance",440,1042,/bnWxRJHWbYcAGq9HWWuKCwm7I5X.jpg,


### Fetch show metadata

In [1284]:
url = 'https://graphql.anilist.co'

query = '''
query ($id: Int, $page: Int, $perPage: Int, $search: String) {
    Page (page: $page, perPage: $perPage) {
        pageInfo {
            total
            currentPage
            lastPage
            hasNextPage
            perPage
        }
        media (id: $id, search: $search, type: ANIME) {
            id
            source
            genres
            episodes
            title {
                romaji
            }


            studios {
              edges {
                id
                isMain
                node {
                  name
                }
              }
            }


        }
    }
}
'''

In [1285]:
# Query Anilist
cdf['Genres'].astype(object) # to insert lists
cdf = cdf.replace(np.nan,"")

def AniQuery(cdf,variables,title,query=query):
    response = requests.post(url, json={'query': query, 'variables': variables})
    parsed = json.loads(response.text)
    dat = parsed['data']['Page']['media'][0]
    #pretty_data = json.dumps(dat, indent=4, sort_keys=True)
    #print(pretty_data,'\n')

    cdf.loc[title,'Source'] = dat['source']
    if dat['studios']['edges']:
        cdf.loc[title,'Studio'] = list(filter(lambda x:x["isMain"]==True,dat['studios']['edges']))[0]['node']['name']
    cdf.at[title, 'Genres'] = dat['genres']

    cdf.loc[title, 'AniListID'] = dat['id'] 

for i, title in enumerate(cdf.index):

    if i % 50 == 0:
        print(i,title)
    variables = {'search': title}
    
    if (cdf.loc[title, 'Source'] in [np.nan,None, ''] or
        cdf.loc[title, 'Genres'] in [np.nan,None, ''] or
        cdf.loc[title, 'Studio'] in [np.nan,None, ''] or
        cdf.loc[title, 'AniListID'] in [np.nan,None, '']):

        try:
            if cdf.loc[title, 'AniListID'] != "":
              variables = {"id": cdf.loc[title, 'AniListID']}
            AniQuery(cdf,variables,title)
        except KeyError:
            try:
                variables = {'search': cdf.loc[title,'AltTitle']}
                AniQuery(cdf,variables,title)
            except KeyError:
                print('KeyError:',title)
                continue
        except IndexError:
            try:
                variables = {'search': cdf.loc[title,'AltTitle']}
                AniQuery(cdf,variables,title)
            except IndexError:
                print('IndexError:', title, cdf.loc[title,'AltTitle'])
                continue

            
cdf.head()

0 Neon Genesis Evangelion
50 Death Note
100 Bunny Drop
IndexError: Gutsy Frog 
150 Kids on the Slope
200 Slayers
250 Strawberry Marshmallow
IndexError: Japan Animator Expo 
IndexError: Cat Soup Theater 
300 7 Seeds
IndexError: Doraemon 
350 Rainbow
IndexError: There She Is!! 
400 Air
IndexError: Haré+Guu 
IndexError: Guardian Ninja Mamoru Protect from the Shadows!
450 Space Runaway Ideon
IndexError: Manyuu Hikenchou 


,Rank,Votes,Score,Esoteric\nRank,Esoteric\nScore,Source,Episodes,Studio,First Air Date,Last Air Date,Genres,AniListID,TMDbID,IMGID,AltTitle
Title,,,,,,,,,,,,,,,
Neon Genesis Evangelion,1,61,1374.0,148,18.1,Original,26,Gainax,1995-10-04,1996-03-27,"Action, Drama, Mecha, Mystery, Psychological, ...",30,890,/y2ah9t0navXyIvoHg1uIbIHO3tt.jpg,
Serial Experiments Lain,2,49,1112.0,184,12.4,Original,13,Triangle Staff,1998-07-06,1998-09-28,"Drama, Mystery, Psychological, Sci-Fi, Superna...",339,1087,/ej3tcxv2YYVWy6WoOeWZTcrkiI8.jpg,
Cowboy Bebop,3,46,978.0,188,12.3,Original,26,Sunrise,1998-04-03,1999-04-24,"Action, Adventure, Drama, Sci-Fi",1,30991,/xDiXDfZwC6XYC6fxHI1jl3A3Ill.jpg,
FLCL,4,37,785.0,226,8.5,Original,6,Gainax,2003-05-05,2003-05-13,"Action, Comedy, Mecha, Sci-Fi",227,5895,/FkgA8CcmiLJGVCRYRQ2g2UfVtF.jpg,
Revolutionary Girl Utena,5,30,708.0,193,12.1,Original,39,J.C. Staff,1997-04-02,1997-12-24,"Drama, Fantasy, Mystery, Psychological, Romance",440,1042,/bnWxRJHWbYcAGq9HWWuKCwm7I5X.jpg,


In [1286]:
# Flatten genres list
separator = ', '
for title in cdf.index:
    if type(cdf.loc[title,'Genres']) == list:
        cdf.loc[title,'Genres'] = separator.join(cdf.loc[title,'Genres'])


# Capitalize Source
cdf['Source'] = cdf['Source'].str.replace('_',' ')
cdf['Source'] = cdf['Source'].str.title()
        
cdf.tail()

,Rank,Votes,Score,Esoteric\nRank,Esoteric\nScore,Source,Episodes,Studio,First Air Date,Last Air Date,Genres,AniListID,TMDbID,IMGID,AltTitle
Title,,,,,,,,,,,,,,,
Dragon Ball Super,458,1,1.0,266,4.9,Manga,131,Toei Animation,2015-07-05,2018-03-25,"Action, Adventure, Comedy",21175,62715,/kbhu0tIv5E1PbAw2YPjrhoQ7I1s.jpg,
Galaxy Express 999,458,1,1.0,378,1.0,Manga,113,Toei Animation,1978-09-14,1981-03-12,"Adventure, Drama, Mecha, Sci-Fi",1491,31709,/hSL8OUYK1TRKENMgrgStaY5lqzK.jpg,
Tonkatsu DJ Ageratou,458,1,1.0,365,1.1,Manga,,Studio DEEN,,,"Comedy, Music",21297,,,
Space Adventure Cobra!,458,1,1.0,378,1.0,Manga,31,Tokyo Movie Shinsha,1982-10-07,1983-05-19,"Action, Adventure, Sci-Fi",2451,8880,/lsocXzEZv1fqUN5zVqptMvIUEMy.jpg,
Ikki Tousen,458,1,1.0,359,1.2,Manga,49,J.C. Staff,2003-07-30,2010-06-11,"Action, Ecchi",257,37584,/eV0cphNP9cyYL28Rxcs1C3lbEGj.jpg,


In [1287]:
# Query TMDb.org

f = open("/content/drive/MyDrive/RYMAnimeChart/tmdb_api_key.txt", "r")
api_key = f.read()[:-1]
f.close()

for i, title in enumerate(cdf.index):

    if i % 50 == 0:
        print(i,title)
    
    if (cdf.loc[title,'Episodes'] in [None, ''] or
        cdf.loc[title,'First Air Date'] in [None, ''] or
        cdf.loc[title,'Last Air Date'] in [None, ''] or 
        cdf.loc[title, 'IMGID'] in [None,''] or 
        cdf.loc[title, 'TMDbID'] in [None,'']):
        
        title_id = str(cdf.loc[title, 'TMDbID'])
        
        try:
            if title_id in [None,'','None']:
                
                r = requests.get('https://api.themoviedb.org/3/search/tv?api_key='+api_key+'&query='+title)
                parsed = json.loads(r.text)  
                
                # Make sure TMDb genre contains animation (possibly: origin country is JP)
                for j, res in enumerate(parsed['results']):
                    if 16 in res['genre_ids']: # TMDb genre ID for animation = 16
                        title_id = str(res['id'])
                        cdf.loc[title, 'TMDbID'] = title_id
                        break

                if title_id in [None,'','None']:
                    raise NameError('TitleID')

        except KeyError:
            print('KeyError1:',title)
            continue
        except IndexError:
            print('IndexError1:',title)
            continue
        except NameError:
            print('NameError1',title)
            continue     

        # Get episodes
        r = requests.get('https://api.themoviedb.org/3/tv/'+title_id+'?api_key='+api_key)
        parsed = json.loads(r.text)
        #pretty_data = json.dumps(parsed, indent=4, sort_keys=True)
        #print(pretty_data)
        
        try:
            cdf.loc[title, 'IMGID'] = parsed['poster_path']
            cdf.loc[title, 'Episodes'] = parsed['number_of_episodes']
            cdf.loc[title, 'First Air Date'] = parsed['first_air_date']
            cdf.loc[title, 'Last Air Date'] = parsed['last_air_date']
        except KeyError:
            print('KeyError2:',title,title_id)
            if title_id in [np.nan]:
              print("erere")
            print(type(title_id))
            continue
        except IndexError:
            print('IndexError2:',title)
            continue
        except NameError:
            print('NameError2',title)
            continue     

0 Neon Genesis Evangelion
50 Death Note
100 Bunny Drop
150 Kids on the Slope
NameError1 Touch
200 Slayers
250 Strawberry Marshmallow
NameError1 Tetsujin 28-Gou
NameError1 Manga Children's Library
300 7 Seeds
NameError1 Dirty Pair (1985)
NameError1 Ratio of Forest
350 Rainbow
NameError1 The Diary of Tortov Roddle
NameError1 Neighborhood Stories
NameError1 Crayon Kingdom of Dreams
400 Air
NameError1 Onegai My Melody
NameError1 Urotsukidoji
NameError1 Invincible Superman Zambot 3
450 Space Runaway Ideon
NameError1 Tonkatsu DJ Ageratou


In [1288]:
cols = ["Rank",	"Votes",	"Score",	"Esoteric\nRank",	"Esoteric\nScore",	"Source",	"Episodes",	"Studio",	"First Air Date",	"Last Air Date",	"Genres",	"AniListID",	"TMDbID",	"IMGID"]
cdf = cdf[cols]
set_with_dataframe(chartsheet, cdf.sort_values("Rank"), include_index=True)

In [1289]:
print("Shows:", len(cdf))
print("Voters:", len(vdf.columns))
print("Votes:", np.sum(cdf["Votes"]))
print("Avg. list length:", np.sum(cdf["Votes"])/len(vdf.columns))

Shows: 472
Voters: 80
Votes: 2010
Avg. list length: 25.125
